In [49]:
import pandas as pd
import datetime
import re

In [110]:
today = datetime.datetime.today()
dateToday = today.strftime("%Y-%m-%d")
print(dateToday)


2019-11-04


In [35]:
log = pd.read_csv("FutArb_trading_log.csv", sep='\s*,\s*')
print(log)

   ï»¿Strategy                              Time Contract      conId  \
0       FutArb  2019-09-12 13:32:04.733362+00:00       ES  321454763   
1       FutArb  2019-09-12 13:32:05.614997+00:00      RTY  321454816   
2       FutArb  2019-09-12 13:48:04.979256+00:00       ES  321454763   
3       FutArb  2019-09-12 13:48:05.960527+00:00      RTY  321454816   
4       FutArb  2019-09-13 13:32:04.254756+00:00       ES  321454763   
..         ...                               ...      ...        ...   
64      FutArb  2019-10-17 14:00:02.995833+00:00      RTY  334144674   
65      FutArb  2019-10-28 14:25:03.074362+00:00       ES  334144679   
66      FutArb  2019-10-28 14:25:03.678424+00:00      RTY  334144674   
67      FutArb  2019-10-28 15:00:01.329244+00:00       ES  334144679   
68      FutArb  2019-10-28 15:00:02.256349+00:00      RTY  334144674   

      Expiry Direction  Exec price  Quantity  
0   20190920       Buy     3010.25       1.0  
1   20190920      Sell     1573.30       

c:\users\auria\onedrive\desktop\computer science\ibtrading\trading-env\lib\site-packages\ipykernel_launcher.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  """Entry point for launching an IPython kernel.


In [143]:
def buildFutarbPnl(date):
#Create trade recap and P&L

    futarb_report = None

    for row in log.iterrows():
        logDate = row[1]['Time']
        match = re.search(r'\d{4}-\d{2}-\d{2}', logDate)
        logDate = match.group()

        if logDate == date:
            #create table entries
            contract = row[1]['Contract']
            direction = row[1]['Direction']
            quantity = row[1]['Quantity']
            price = row[1]['Exec price']
            if direction == 'Buy':
                cashFlow = -1 * price * quantity * 50
            else:
                cashFlow = 1 * price * quantity * 50

            #create list if it doesn't exist yet
            if futarb_report == None:
                futarb_report = [[contract, direction, quantity, price, cashFlow]]
            else:
                futarb_report.append([contract, direction, quantity, price, cashFlow])

    #create dataframe
    futarb_dataframe = pd.DataFrame(futarb_report, columns = ['contract', 'direction', 'quantity', 'price', 'cashFlow'])


    futarb_pnl = 0
    for row in futarb_dataframe.iterrows():
        futarb_pnl += row[1]['cashFlow']
        
    return futarb_dataframe, futarb_pnl

            

    

In [144]:
def buildFutarbHistoricalPnl():
#build historical PnL
    historicalReport = None
    buildingDate = None

    for row in log.iterrows():
        logDate = row[1]['Time']
        match = re.search(r'\d{4}-\d{2}-\d{2}', logDate)
        logDate = match.group()

        if logDate != buildingDate:
            buildingDate = logDate
            trades, pnl = buildFutarbPnl(logDate)

            if historicalReport == None:
                historicalReport = [[logDate, pnl]]
            else:
                historicalReport.append([logDate, pnl])
    futarbHistoricalDataframe = pd.DataFrame(historicalReport, columns = ['Date', 'P&L'])
    return futarbHistoricalDataframe





In [145]:
def calculateMetrics(pnl):
    entries = pnl['P&L']
    averageReturn = pnl['P&L'].mean()
    totalReturn = pnl['P&L'].sum()
    entries = 0
    positives = 0
    
    for row in pnl.iterrows():
        value = row[1]['P&L']
        entries += 1
        if value > 0:
            positives +=1

    hitRatio = positives / entries
    
        
    
    
    return averageReturn, totalReturn, hitRatio
    
        
    

In [146]:
table = buildFutarbHistoricalPnl()
print('Since inception: ')
print(table.to_string(index=False))
print('')

table = buildFutarbHistoricalPnl()
averageReturn, totalReturn, hitRatio = calculateMetrics (table)
print('Average return = ' + "{:.{}f}".format( averageReturn, 2 ))
print('Total return = ' + "{:.{}f}".format( totalReturn, 2 ))
print('Hit Ratio = ' + '{percent:.2%}'.format(percent=hitRatio))
print('')

print('Daily P&L:')
date = '2019-10-16'
dailyTrades, dailyPNL = buildFutarbPnl(date)

print(dailyTrades)
print('P&L today: ' + str(dailyPNL))


Since inception: 
       Date     P&L
 2019-09-12   602.5
 2019-09-13   575.0
 2019-09-16   472.5
 2019-09-17   -55.0
 2019-09-18    55.0
 2019-09-19  -135.0
 2019-09-20  5602.5
 2019-09-24    77.5
 2019-09-26  -197.5
 2019-10-03   215.0
 2019-10-04   -92.5
 2019-10-07    97.5
 2019-10-09   -65.0
 2019-10-15  -375.0
 2019-10-16   237.5
 2019-10-17   245.0
 2019-10-28    12.5

Average return = 427.79
Total return = 7272.50
Hit Ratio = 64.71%

Daily P&L:
  contract direction  quantity    price  cashFlow
0       ES      Sell       1.0  2986.25  149312.5
1      RTY       Buy       2.0  1522.10 -152210.0
2       ES       Buy       1.0  2987.50 -149375.0
3      RTY      Sell       2.0  1525.10  152510.0
P&L today: 237.5
